XGBoost implementation of the half life regression baseline model

In [1]:
import math
import os
import random
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from scipy.stats import spearmanr
import shap
import matplotlib.pyplot as plt
from sys import intern
import optuna

from collections import defaultdict, namedtuple

c:\Users\juanm\miniconda3\envs\kulhack\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# various constraints on parameters and outputs
from numpy.ma import mean


MIN_HALF_LIFE = 15.0 / (24 * 60)    # 15 minutes
MAX_HALF_LIFE = 274.                # 9 months
LN2 = math.log(2.)


In [3]:
def get_xgboost_data(input_file, max_lines=None):
    print(f"Reading data from {input_file}...")
    
    # 1. Load Data
    if max_lines is not None:
        df = pd.read_csv(input_file, compression='infer', nrows=max_lines)
    else:
        df = pd.read_csv(input_file, compression='infer')

    df['pos_tag'] = df['lexeme_string'].str.extract(r'<([^>]+)>').fillna('unknown')
    df['historical_accuracy'] = np.where(
        df['history_seen'] > 0, 
        df['history_correct'] / df['history_seen'], 
        0.0
    )
    
    # 5. Global User Accuracy
    user_acc = df.groupby('user_id').apply(
        lambda x: x['history_correct'].sum() / (x['history_seen'].sum() + 1e-5)
    ).reset_index(name='user_global_accuracy')
    df = df.merge(user_acc, on='user_id', how='left')
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit='s')
    
    # 2. Vectorized Target Variable (y)
    # Replaces the pclip() function by clipping the entire column at once
    y = df['p_recall'].clip(lower=0.0001, upper=0.9999)

    # 3. Vectorized Feature Engineering (X)
    X = pd.DataFrame()


    X['hour_of_day'] = df['timestamp'].dt.hour
    X['day_of_week'] = df['timestamp'].dt.dayofweek

    X['time_lag_days'] = df['delta'] / (60 * 60 * 24)
    X['right'] = np.sqrt(1 + df['history_correct'])
    X['wrong'] = np.sqrt(1 + (df['history_seen'] - df['history_correct']))
    df['log_delta'] = np.log1p(df['delta'] / (60 * 60 * 24))
    # Combine languages into a single feature
    X['lang'] = df['ui_language'] + "->" + df['learning_language']

    X['historical_accuracy'] = df['historical_accuracy']
    X['log_delta'] = df['log_delta']
    
    # 4. The XGBoost Categorical Magic
    # Cast strings to pandas categorical dtype. XGBoost will handle the rest!
    X['lang'] = X['lang'].astype('category')
    #X['lexeme_string'] = df['lexeme_string'].astype('category')
    X['pos_tag'] = df['pos_tag'].astype('category')

    print("Data processing complete! Splitting data...")
    
    # 5. Fast 90/10 Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    
    return X_train, X_test, y_train, y_test

In [ ]:
def train_xgboost_baseline(X_train, X_test, y_train, y_test):
    print("Initializing XGBoost Regressor (Phase 2 Baseline)...")
    
    # 1. Target Transformation: Calculate 'h' for training
    # We use X_train['time_lag_days'] for 't'
    h_train = -X_train['time_lag_days'] / np.log2(y_train)
    h_train = np.clip(h_train, MIN_HALF_LIFE, MAX_HALF_LIFE)
    
    h_test = -X_test['time_lag_days'] / np.log2(y_test)
    h_test = np.clip(h_test, MIN_HALF_LIFE, MAX_HALF_LIFE)
    
    # 1. The Model Architecture
    # tree_method="hist" is required for native categorical support
    # We set a high n_estimators but use early_stopping to prevent overfitting
    model = xgb.XGBRegressor(
        tree_method="hist", 
        enable_categorical=True,
        n_estimators=1000,
        learning_rate=0.001,
        max_depth=6,
        early_stopping_rounds=50,
        random_state=42
    )

    print("Training model... (Monitor validation loss to prevent overfitting)")
    
    # 2. The Training Loop
    # We pass the test set as the eval_set so the model can track its out-of-sample error
    model.fit(
        X_train, h_train,
        eval_set=[(X_train, h_train), (X_test, h_test)],
        verbose=50 # Prints update every 50 trees
    )

    # 2. Predict h
    h_pred = model.predict(X_test)
    h_pred = np.clip(h_pred, MIN_HALF_LIFE, MAX_HALF_LIFE)
    
    # 3. Transform h back to probability of recall (p)
    p_pred = 2.0 ** (-X_test['time_lag_days'] / h_pred)
    p_pred = np.clip(p_pred, 0.0001, 0.9999)

    # 4. Evaluate against the ORIGINAL p_recall 
    mae_h = mean_absolute_error(h_test, h_pred)
    spearman_h, _ = spearmanr(h_test, h_pred)
    
    # --- Probability of Recall (p) Metrics ---
    mae_p = mean_absolute_error(y_test, p_pred)
    spearman_p,_ = spearmanr(y_test, p_pred)

    print("-" * 30)
    print("🏆 PHASE 2 RESULTS 🏆")
    print(f"  MAE (Days):           {mae_h:.4f}")
    print(f"  Spearman Correlation: {spearman_h:.4f}")
    print("-" * 40)
    print("RECALL PROBABILITY (p) PREDICTION:")
    print(f"  MAE:                  {mae_p:.4f}")
    print(f"  Spearman Correlation: {spearman_p:.4f}")
    print("-" * 30)
    
    return model

In [ ]:
def optimize_xgboost(X_train_full, y_train_full_p):
    print("Initializing Optuna Hyperparameter Hunt...")
    
    # Split training data to create a dedicated validation set for Optuna
    # This prevents us from overfitting to our final holdout test set!
    X_train_opt, X_val_opt, y_train_opt_p, y_val_opt_p = train_test_split(
        X_train_full, y_train_full_p, test_size=0.2, random_state=42
    )
    
    h_train_opt = np.clip(-X_train_opt['time_lag_days'] / np.log2(y_train_opt_p), MIN_HALF_LIFE, MAX_HALF_LIFE)
    h_val_opt = np.clip(-X_val_opt['time_lag_days'] / np.log2(y_val_opt_p), MIN_HALF_LIFE, MAX_HALF_LIFE)

    def objective(trial):
        # 1. Define the search space
        params = {
            "tree_method": "hist",
            "enable_categorical": True,
            "n_estimators": 500, # Keep lower for faster hackathon iteration
            "learning_rate": trial.suggest_float("learning_rate", 0.001, 0.2, log=True),
            "max_depth": trial.suggest_int("max_depth", 4, 10),
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
            "subsample": trial.suggest_float("subsample", 0.6, 1.0),
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
            "random_state": 42
        }

        # 2. Train the model
        model = xgb.XGBRegressor(**params)
        model.fit(
            X_train_opt, h_train_opt,
            eval_set=[(X_val_opt, h_val_opt)],
            verbose=False # Keep terminal clean
        )

        # 3. Predict h and transform back to p_recall
        h_pred = np.clip(model.predict(X_val_opt), MIN_HALF_LIFE, MAX_HALF_LIFE)
        p_pred = np.clip(2.0 ** (-X_val_opt['time_lag_days'] / h_pred), 0.0001, 0.9999)

        # 4. Return the metric we want to minimize
        return mean_absolute_error(y_val_opt_p, p_pred)

    # Run the study
    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=30) # 30 trials is a good hackathon sweet spot

    print("\n🏆 OPTUNA SEARCH COMPLETE 🏆")
    print("Best Trial MAE:", study.best_value)
    print("Best Parameters:", study.best_params)
    
    return study.best_params

In [6]:
X_train, X_test, y_train, y_test = get_xgboost_data("../data/SpacedRepetitionData.csv")
baseline_model = train_xgboost_baseline(X_train, X_test, y_train, y_test)
baseline_model.save_model("xgboost_baseline.json")

Reading data from ../data/SpacedRepetitionData.csv...
Data processing complete! Splitting data...
Initializing XGBoost Regressor (Phase 2 Baseline)...
Training model... (Monitor validation loss to prevent overfitting)
[0]	validation_0-rmse:116.83664	validation_1-rmse:116.82482
[50]	validation_0-rmse:79.05515	validation_1-rmse:79.00379
[100]	validation_0-rmse:79.00607	validation_1-rmse:78.96547
[150]	validation_0-rmse:78.97959	validation_1-rmse:78.95420
[200]	validation_0-rmse:78.96053	validation_1-rmse:78.94784
[250]	validation_0-rmse:78.94333	validation_1-rmse:78.94356
[300]	validation_0-rmse:78.92935	validation_1-rmse:78.94199
[350]	validation_0-rmse:78.91467	validation_1-rmse:78.94114
[400]	validation_0-rmse:78.90098	validation_1-rmse:78.93942
[450]	validation_0-rmse:78.88664	validation_1-rmse:78.93801
[500]	validation_0-rmse:78.87450	validation_1-rmse:78.93647
[550]	validation_0-rmse:78.86218	validation_1-rmse:78.93630
[564]	validation_0-rmse:78.85936	validation_1-rmse:78.93659
---

In [ ]:
baseline_model = train_xgboost_baseline(X_train, X_test, y_train, y_test)
